In [3]:
import time
import math
import random
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use("dark_background")

import os
os.chdir("/home/cc/proformer_ce/")

In [5]:
df = pd.read_csv("data/finale.csv")
df = df.rename(columns={"Case ID": "case_id", "Activity": "activity", 
                        "Resource": "resource", "Complete Timestamp": "timestamp",
                        "Variant": "variant", "Variant index": "variant_id"})
df.head()

,case_id,activity,resource,timestamp,variant,variant_id,Variant.1,seriousness,customer,product,responsible_section,seriousness_2,service_level,service_type,support_section,workgroup
0,Case 1,Assign seriousness,Value 1,2012/10/09 14:50:17.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
1,Case 1,Take in charge ticket,Value 1,2012/10/09 14:51:01.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
2,Case 1,Take in charge ticket,Value 2,2012/10/12 15:02:56.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
3,Case 1,Resolve ticket,Value 1,2012/10/25 11:54:26.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
4,Case 1,Closed,Value 3,2012/11/09 12:54:39.000,Variant 12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1


In [6]:
df.case_id = df.case_id.str.removeprefix("Case ").astype(int)
df.resource = df.resource.str.removeprefix("Value ").astype(int)
df.variant = df.variant.str.removeprefix("Variant ").astype(int)
df

,case_id,activity,resource,timestamp,variant,variant_id,Variant.1,seriousness,customer,product,responsible_section,seriousness_2,service_level,service_type,support_section,workgroup
0,1,Assign seriousness,1,2012/10/09 14:50:17.000,12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
1,1,Take in charge ticket,1,2012/10/09 14:51:01.000,12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
2,1,Take in charge ticket,2,2012/10/12 15:02:56.000,12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
3,1,Resolve ticket,1,2012/10/25 11:54:26.000,12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
4,1,Closed,3,2012/11/09 12:54:39.000,12,12,Variant 12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21343,4579,Closed,5,2010/09/02 10:11:00.000,1,1,Variant 1,Value 1,Value 71,Value 3,Value 1,Value 1,Value 3,Value 1,Value 1,Value 1
21344,4580,Take in charge ticket,6,2012/01/03 09:33:43.000,18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21345,4580,Wait,6,2012/01/10 15:30:11.000,18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21346,4580,Resolve ticket,6,2012/01/10 17:07:40.000,18,18,Variant 18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1


In [8]:
df.to_csv("data/finale_preprocessed.csv", index=False)

In [10]:
df.groupby("case_id").apply(lambda x : len(x.activity)).max()

15